In [2]:
import numpy as np
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import KFold

In [3]:
! pip install pyriemann

  Retrying (Retry(total=4, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f2e2dd02390>: Failed to establish a new connection: [Errno -2] Name or service not known',)': /simple/pyriemann/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f2e2dd02c50>: Failed to establish a new connection: [Errno -2] Name or service not known',)': /simple/pyriemann/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f2e2dd029b0>: Failed to establish a new connection: [Errno -2] Name or service not known',)': /simple/pyriemann/
  Retrying (Retry(total=1, connect=None, read=None, 

In [4]:
! pip install pyriemann --user

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
pip install pyriemann--upgrade pip


The following command must be run outside of the IPython shell:

    $ pip install pyriemann--upgrade pip

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [6]:
! pip install pyriemann --upgrade

Requirement already up-to-date: pyriemann in /home/jatin/anaconda3/lib/python3.6/site-packages
    100% |████████████████████████████████| 25.9MB 26kB/s eta 0:00:011
    100% |████████████████████████████████| 13.5MB 42kB/s eta 0:00:011
Requirement already up-to-date: joblib in /home/jatin/anaconda3/lib/python3.6/site-packages (from pyriemann)
    100% |████████████████████████████████| 6.9MB 106kB/s ta 0:00:011    52% |████████████████▉               | 3.6MB 553kB/s eta 0:00:06
    100% |████████████████████████████████| 10.1MB 75kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4f/a4/879454d49688e2fad93e59d7d4efda580b783c745fd2ec2a3adf87b0808d/pytz-2020.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-

In [7]:
import numpy as np 
from scipy.special import binom
import pyriemann.utils.mean as rie_mean
from filters import butter_fir_filter
from eig import gevd

ModuleNotFoundError: No module named 'filters'

In [8]:
import numpy as np
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import KFold

In [10]:
import import_ipynb
from csp import generate_projection,generate_eye,extract_feature
from get_data import get_data
from filters import load_filterbank 

importing Jupyter notebook from csp.ipynb
importing Jupyter notebook from filters.ipynb
importing Jupyter notebook from eig.ipynb
importing Jupyter notebook from get_data.ipynb


In [12]:
class CSP_Model:
    def __init__(self):
        self.crossvalidation = False
        self.data_path 	= 'dataset/'
        self.svm_kernel	= 'linear' #'sigmoid'#'linear' # 'sigmoid', 'rbf', 'poly'
        self.svm_c 	= 0.1 # 0.05 for linear, 20 for rbf, poly: 0.1
        self.useCSP = True
        self.NO_splits = 5 # number of folds in cross validation 
        self.fs = 250. # sampling frequency 
        self.NO_channels = 22 # number of EEG channels 
        self.NO_subjects = 9
        self.NO_csp = 24 # Total number of CSP feature per band and timewindow
        self.bw = np.array([2,4,8,16,32]) # bandwidth of filtered signals 
        # self.bw = np.array([1,2,4,8,16,32])
        self.ftype = 'butter' # 'fir', 'butter'
        self.forder= 2 # 4
        self.filter_bank = load_filterbank(self.bw,self.fs,order=self.forder,max_freq=40,ftype = self.ftype)
        time_windows_flt = np.array([
                                        [2.5,3.5],
                                        [3,4],
                                        [3.5,4.5],
                                        [4,5],
                                        [4.5,5.5],
                                        [5,6],
                                        [2.5,4.5],
                                        [3,5],
                                        [3.5,5.5],
                                        [4,6],
                                        [2.5,6]])*self.fs # time windows in [s] x fs for using as a feature
        self.time_windows = time_windows_flt.astype(int)
        self.NO_bands = self.filter_bank.shape[0]
        self.NO_time_windows = int(self.time_windows.size/2)
        self.NO_features = self.NO_csp*self.NO_bands*self.NO_time_windows
        self.train_time = 0
        self.train_trials = 0
        self.eval_time = 0
        self.eval_trials = 0
    def run_csp(self):
        ################################ Training ############################################################################
        start_train = time.time()
        # 1. Apply CSP to bands to get spatial filter 
        if self.useCSP: 
            w = generate_projection(self.train_data,self.train_label, self.NO_csp,self.filter_bank,self.time_windows)
        else: 
            w = generate_eye(self.train_data,self.train_label,self.filter_bank,self.time_windows)


        # 2. Extract features for training 
        feature_mat = extract_feature(self.train_data,w,self.filter_bank,self.time_windows)

        # 3. Stage Train SVM Model 
        # 2. Train SVM Model 
        if self.svm_kernel == 'linear' : 
            clf = LinearSVC(C = self.svm_c, intercept_scaling=1, loss='hinge', max_iter=1000,multi_class='ovr', penalty='l2', random_state=1, tol=0.00001)
        else:
            clf = SVC(self.svm_c,self.svm_kernel, degree=10, gamma='auto', coef0=0.0, tol=0.001, cache_size=10000, max_iter=-1, decision_function_shape='ovr')
        clf.fit(feature_mat,self.train_label) 

        end_train = time.time()
        self.train_time += end_train-start_train
        self.train_trials += len(self.train_label)
        ################################# Evaluation ###################################################
        start_eval = time.time()
        eval_feature_mat = extract_feature(self.eval_data,w,self.filter_bank,self.time_windows)

        success_rate = clf.score(eval_feature_mat,self.eval_label)

        end_eval = time.time()
        
        #print("Time for one Evaluation " + str((end_eval-start_eval)/len(self.eval_label)) )

        self.eval_time += end_eval-start_eval
        self.eval_trials += len(self.eval_label)


        return success_rate 
    def load_data(self):
            if self.crossvalidation:
                data,label = get_data(self.subject,True,self.data_path)
                kf = KFold(n_splits=self.NO_splits)
                split = 0 
                for train_index, test_index in kf.split(data):
                    if self.split == split:
                        self.train_data = data[train_index]
                        self.train_label = label[train_index]
                        self.eval_data = data[test_index]
                        self.eval_label = label[test_index]
                    split += 1
            else:
                self.train_data,self.train_label = get_data(self.subject,True,self.data_path)
                self.eval_data,self.eval_label = get_data(self.subject,False,self.data_path)

In [13]:
def main():
    model = CSP_Model()

    print("Number of used features: "+ str(model.NO_features))

    # success rate sum over all subjects 
    success_tot_sum = 0

    if model.crossvalidation:
        print("Cross validation run")
    else: 
        print("Test data set")
    start = time.time()
    
    for model.subject in range(1,model.NO_subjects+1):

        #print("Subject" + str(model.subject)+":")
        

        if model.crossvalidation:
            success_sub_sum = 0 

            for model.split in range(model.NO_splits):
                model.load_data()
                success_sub_sum += model.run_csp()
                print(success_sub_sum/(model.split+1))
            # average over all splits 
            success_rate = success_sub_sum/model.NO_splits

        else: 
            # load Eval data 
            model.load_data()
            success_rate = model.run_csp()

        print(success_rate)
        success_tot_sum += success_rate 
        
        # Average success rate over all subjects 
    print("Average success rate: " + str(success_tot_sum/model.NO_subjects))

    print("Training average time: " +  str(model.train_time/model.NO_subjects))
    print("Evaluation average time: " +  str(model.eval_time/model.NO_subjects))

    end = time.time()	

    print("Time elapsed [s] " + str(end - start))

In [15]:
if __name__ == '__main__':
    main()

Number of used features: 11352
Test data set


FileNotFoundError: [Errno 2] No such file or directory: '/home/jatin/BCI project/datasetA01T.mat'